# Training the tensorflow model on Google Cloud AI Platform

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/Breast-Cancer-Detection

In [ ]:
# without this, cv2 throws an error 
!sudo apt-get install ffmpeg libsm6 libxext6  -y

In [3]:
import os
import yaml

In [4]:
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

In [5]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.3'

In [6]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [15]:
os.environ['TRAIN_DIR'] = 'gs://vernal-buffer-285906/train_data'
os.environ['OUTPUT_DIR'] = 'gs://vernal-buffer-285906/train_results'
os.environ['CONFIG_PATH'] = 'Breast-Cancer-Detection/config/config.yaml'

In [ ]:
! gsutil -m cp -r ../../config gs://$BUCKET

## Train locally using python

In [ ]:
%%bash
export PYTHONPATH=${PYTHONPATH}:/home/jupyter/Breast-Cancer-Detection/
#Train using python
python -m detectors.detector \
    --train \
    --train-config=$CONFIG_PATH

## Submit Training job to Google Cloud Ai Platform

In [16]:
%%bash
export PYTHONPATH=${PYTHONPATH}:/home/jupyter/Breast-Cancer-Detection/

JOBNAME=breastcancer_$(date -u +%y%m%d_%H%M%S)
echo $OUTPUT_DIR $REGION $JOBNAME

gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=detectors.tf_gcp.trainer.task \
  --package-path=/home/jupyter/Breast-Cancer-Detection/detectors/ \
  --job-dir=$OUTPUT_DIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=2.3 \
  --python-version=3.7 \
  -- \
  --train-config=gs://$BUCKET/config/config.yaml

gs://vernal-buffer-285906/train_results us-west1 breastcancer_210704_141124
jobId: breastcancer_210704_141124
state: QUEUED


Job [breastcancer_210704_141124] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe breastcancer_210704_141124

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs breastcancer_210704_141124


In [ ]:
# Running TensorBoard
# Open the port you are supposed to use (usually 6006) in Google cloud vm instance
# info -> https://docs.bitnami.com/google/faq/administration/use-firewall/
# Run tensorboard using following command
# tensorboard --logdir <path/to/log/directory> --bind_all
# Ex:
# tensorboard --logdir gs://vernal-buffer-285906/model_results/tensorboard --bind_all